**Want to:**

Fetch frames from two cameras and detect centers of colours

Collect camera calibration from xml file

Triangulate

Find depth of both camera

(Have to do two colllection- and finding centers for cameras, and also find depth for both layers with two different colours)

In [1]:

import numpy as np
import time
import imutils
import cv2
import argparse
from imutils.video import VideoStream
import glob
import matplotlib.pyplot as plt
import os
import re

*Calibrating*

In [ ]:

################ FIND CHESSBOARD CORNERS - OBJECT POINTS AND IMAGE POINTS #############################

chessboardSize = (9,6)
frameSize = (1080,1920)

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)
objp *= (8,7.5,0) #Size of squares in chessboard

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpointsL = [] # 2d points in image plane.
imgpointsR = [] # 2d points in image plane.

imagesLeft = glob.glob('Week16/Calib/left/*.png')
imagesRight = glob.glob('Week16/Calib/right/*.png')

imagesLeft = sorted(imagesLeft)
imagesRight = sorted(imagesRight)

index =0
for imgLeft, imgRight in zip(imagesLeft, imagesRight):

    imgL = cv2.imread(imgLeft)
    imgR = cv2.imread(imgRight)
    grayL = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)
    grayR = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    retL, cornersL = cv2.findChessboardCorners(grayL, chessboardSize, None)
    retR, cornersR = cv2.findChessboardCorners(grayR, chessboardSize, None)

    # If found, add object points, image points (after refining them)
    if retL and retR == True:

        objpoints.append(objp)

        cornersL = cv2.cornerSubPix(grayL, cornersL, (11,11), (-1,-1), criteria)
        imgpointsL.append(cornersL)

        cornersR = cv2.cornerSubPix(grayR, cornersR, (11,11), (-1,-1), criteria)
        imgpointsR.append(cornersR)

        # Draw and display the corners
        cv2.drawChessboardCorners(imgL, chessboardSize, cornersL, retL)
        #cv2.imshow('img left', imgL)
        image_nameL = "Week16/calibres" + '/resL' + str(index) + '.png'
        cv2.imwrite(image_nameL, imgL)
        cv2.drawChessboardCorners(imgR, chessboardSize, cornersR, retR)
        #cv2.imshow('img right', imgR)
        image_nameR = "Week16/calibres" + '/resR' + str(index) + '.png'
        cv2.imwrite(image_nameR, imgL)
        cv2.waitKey(1000)
        index+=1

cv2.destroyAllWindows()

############## CALIBRATION #######################################################

retL, cameraMatrixL, distL, rvecsL, tvecsL = cv2.calibrateCamera(objpoints, imgpointsL, frameSize, None, None)
heightL, widthL, channelsL = imgL.shape
newCameraMatrixL, roi_L = cv2.getOptimalNewCameraMatrix(cameraMatrixL, distL, (widthL, heightL), 1, (widthL, heightL))

retR, cameraMatrixR, distR, rvecsR, tvecsR = cv2.calibrateCamera(objpoints, imgpointsR, frameSize,None, None)
heightR, widthR, channelsR = imgR.shape
newCameraMatrixR, roi_R = cv2.getOptimalNewCameraMatrix(cameraMatrixR, distR, (widthR, heightR), 1, (widthR, heightR))

In [ ]:
########## Stereo Vision Calibration #############################################

flags = 0
flags |= cv2.CALIB_FIX_INTRINSIC

criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
retStereo, newCameraMatrixL, distL, newCameraMatrixR, distR, rot, trans, essentialMatrix, fundamentalMatrix = cv2.stereoCalibrate(objpoints, imgpointsL, imgpointsR, newCameraMatrixL, distL, newCameraMatrixR, distR, grayL.shape[::-1], criteria_stereo, flags)

########## Stereo Rectification #################################################

rectifyScale= 1
rectL, rectR, projMatrixL, projMatrixR, Q, roi_L, roi_R= cv2.stereoRectify(newCameraMatrixL, distL, newCameraMatrixR, distR, grayL.shape[::-1], rot, trans, rectifyScale,(0,0))

stereoMapL = cv2.initUndistortRectifyMap(newCameraMatrixL, distL, rectL, projMatrixL, grayL.shape[::-1], cv2.CV_16SC2)
stereoMapR = cv2.initUndistortRectifyMap(newCameraMatrixR, distR, rectR, projMatrixR, grayR.shape[::-1], cv2.CV_16SC2)

print("Saving parameters!")
cv_file = cv2.FileStorage('stereoMapW16_2.xml', cv2.FILE_STORAGE_WRITE)

cv_file.write('stereoMapL_x',stereoMapL[0])
cv_file.write('stereoMapL_y',stereoMapL[1])
cv_file.write('stereoMapR_x',stereoMapR[0])
cv_file.write('stereoMapR_y',stereoMapR[1])

cv_file.release()


In [ ]:
######Error##########
mean_error = 0
for i in range(len(objpoints)):
 imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecsL[i], tvecsL[i], cameraMatrixL, distL)
 error = cv2.norm(imgpointsL[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
 mean_error += error
 
print( "total error: {}".format(mean_error/len(objpoints)) )

*Collecting calibration*

In [ ]:
# Camera parameters to undistort and rectify images
cv_file = cv2.FileStorage()
cv_file.open('stereoMapW16_2.xml', cv2.FileStorage_READ)

stereoMapL_x = cv_file.getNode('stereoMapL_x').mat()
stereoMapL_y = cv_file.getNode('stereoMapL_y').mat()
stereoMapR_x = cv_file.getNode('stereoMapR_x').mat()
stereoMapR_y = cv_file.getNode('stereoMapR_y').mat()

def undistortRectify(frameR, frameL):
    undistortedL= cv2.remap(frameL, stereoMapL_x, stereoMapL_y, cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
    undistortedR= cv2.remap(frameR, stereoMapR_x, stereoMapR_y, cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
    return undistortedR, undistortedL


imgL = cv2.undistort(imgL, cameraMatrixL, distL, None, newCameraMatrixL)

plt.imshow(imgL)
plt.show()

imgR = cv2.undistort(imgR, cameraMatrixR, distR, None, newCameraMatrixR)

plt.imshow(imgR)
plt.show()

undistR,undistL = undistortRectify(imgR,imgL)
plt.imshow(undistR)
plt.show()
plt.imshow(undistL)

print(undistR.shape)

*Triangulation*

In [ ]:
def find_depth(right_point, left_point, frame_right, frame_left, baseline,f, alpha):

    # CONVERT FOCAL LENGTH f FROM [mm] TO [pixel]:
    height_right, width_right, depth_right = frame_right.shape
    height_left, width_left, depth_left = frame_left.shape

    if width_right == width_left:
        f_pixel = (width_right * 0.5) / np.tan(alpha * 0.5 * np.pi/180)

    else:
        print('Left and right camera frames do not have the same pixel width')
    x_right, _ = right_point[0]  # Extract x-coordinate from tuple
    x_left, _ = left_point[0]  # Extract x-coordinate from tuple

    # CALCULATE THE DISPARITY:
    disparity = x_left-x_right      #Displacement between left and right frames [pixels]

    # CALCULATE DEPTH z:
    zDepth = (baseline*f_pixel)/disparity             #Depth in [cm]

    return zDepth

In [ ]:
#Semi-global matching (SGBM/SGM) by Kevin Wood on youtube or kevinwoddrobotics on github
class DepthMap:
    def __init__(self,showImages):
        root = os.getcwd()
        imgLeftPath = os.path.join(root,"Week16/Calib/left/framel0.png")
        imgRightpath = os.path.join(root,"Week16/Calib/right/framer0.png")
        self.imgLeft = cv2.imread(imgLeftPath,cv2.IMREAD_GRAYSCALE)
        self.imgRight = cv2.imread(imgRightpath,cv2.IMREAD_GRAYSCALE)
        
        if showImages:
            plt.figure()
            plt.subplot(121)
            plt.imshow(self.imgLeft)
            plt.subplot(122)
            plt.imshow(self.imgRight)
            plt.show()

    def computedepthmapBM(self):
        ndispfactor = 12 #adjust
        stereo = cv2.StereoBM.create(numDisparities=16*ndispfactor,blockSize=21)
        disparity = stereo.compute(self.imgLeft,self.imgRight)
        plt.imshow(disparity,"gray")
        plt.show()

    def computedepthmapSGBM(self):
        windowsize = 7
        min_disp = 16
        ndispfactor = 14
        num_disp = 16*ndispfactor-min_disp
        stereo = cv2.StereoSGBM_create(minDisparity=min_disp,numDisparities=num_disp,blockSize=windowsize,P1 = 8*3*windowsize**2,P2=32*3*windowsize**2,disp12MaxDiff=1,uniquenessRatio=15,speckleWindowSize=0,speckleRange=2,preFilterCap=63,mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY)
        
        disparity = stereo.compute(self.imgLeft,self.imgRight).astype(np.float32)/16
        plt.imshow(disparity)
        plt.colorbar()
        plt.show()

def demoStereoBM():
    dp = DepthMap(showImages=False)
    dp.computedepthmapBM()

def demoStereoSGBM():
    dp = DepthMap(showImages=False)
    dp.computedepthmapSGBM()

if __name__ == "__main__":
    demoStereoBM()
    demoStereoSGBM()

In [ ]:
img_height = 1920
real_heightobj = 3.4 #mm
f = 3.67
obj_heightpx = 44 #px from GIMP

def find_depth2(real_heightobj, img_height,f,obj_heightpx):
    sensorh = 1/2.88 #inches = (1inch = 25.4mm)
    #1px = 0.2645833333mm
    sensord = 16 * sensorh
    h = 9*np.sqrt(sensord**2 / 337)
    d = (f * real_heightobj * img_height)/(obj_heightpx * h)
    return d

d = find_depth2(img_height,real_heightobj,f,obj_heightpx)

print("The object is: ",d/10,"cm away from the camera")

*Functions*

In [ ]:
#Define functions
def masker(frame, lower, upper):
    mask = cv2.inRange(frame, lower, upper)
    mask = cv2.dilate(mask, None, iterations=2)
    return mask

def calculate_centers(contours):
    return [(x + w // 2, y + h // 2) for contour in contours for x, y, w, h in map(cv2.boundingRect, contours)]

def filter_centers(centers, colour,side,limit, index):
    try: 
        for i, center in enumerate(centers):
            res = f"./Week16/{colour}/res{side}{i}.txt"
            if index == 0:
                with open(res, "w") as file:
                    print(f"{center[0]},{center[1]}", file=file)
            else:
                with open(res, "r") as file:
                    lines = file.readlines()
                if lines:
                    prev_x, prev_y = map(float, lines[-1].strip().split(","))
                    if abs(center[0] - prev_x) <= limit and abs(center[1] - prev_y) <= limit:
                        with open(res, "a") as file:
                            print(f"{center[0]},{center[1]}", file=file)
    except FileNotFoundError:
        pass

def extract_coordinates(content):
    # Define a regular expression pattern to match coordinates
    pattern = r'(\d+),(\d+)'
    
    # Find all occurrences of coordinates in the content
    coordinates = re.findall(pattern, content)
    # Convert the coordinates from strings to integers
    coordinates = [(int(x), int(y)) for x, y in coordinates]

    return coordinates

def displaydepth(frameleft,frameright,centersl,centersr,B,f,alpha):
    if not centersr or not centersl:
        cv2.putText(frameright, "TRACKING LOST", (75,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255),2)
        cv2.putText(frameleft, "TRACKING LOST", (75,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255),2)
        return frameleft,frameright,"error no depth"
    else:
    # Function to calculate depth of object. Outputs vector of all depths in case of several balls.
    # All formulas used to find depth is in video presentaion
        depth = find_depth(centersr, centersl, frameright, frameleft, B, f, alpha)

        cv2.putText(frameright, "TRACKING", (75,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (124,252,0),2)
        cv2.putText(frameleft, "TRACKING", (75,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (124,252,0),2)
        cv2.putText(frameright, "Distance: " + str(round(depth,3)), (200,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (124,252,0),2)
        cv2.putText(frameleft, "Distance: " + str(round(depth,3)), (200,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (124,252,0),2)
        # Multiply computer value with 205.8 to get real-life depth in [cm]. The factor was found manually.
        return frameleft,frameright,depth

depths = []   
def calculate_depths(frame_right,frame_left,baseline,f_pixel,alpha,folder_pathl,folder_pathr):

    def collect_coords(folder_path):
        files = os.listdir(folder_path)
        for file in files:
                file_path = sorted(os.path.join(folder_path, file))
                # Check if the file is a regular file (not a directory)
                if os.path.isfile(file_path):
                    # Extract coordinates from the file
                    coordinates = extract_coordinates(file_path)
                    print(coordinates)
        return coordinates

    left_coords = collect_coords(folder_pathl)
    right_coords = collect_coords(folder_pathr)
    for leftcoord, rightcoord in zip(left_coords, right_coords):
        # Calculate depth for the pair of points
        depth = find_depth(leftcoord, rightcoord, frame_right,frame_left,baseline,f_pixel,alpha)
        # Append the depth to the depths list
        depths.append(depth)

    return depths



*Main*

In [ ]:

def main():
    red_lower = np.array([0, 100, 122], np.uint8)
    red_upper = np.array([180, 255, 255], np.uint8)
    green_lower = np.array([35, 20, 16], np.uint8)
    green_upper = np.array([110, 196, 197], np.uint8)

    
    paused = False
    index = 0

    vl_path = "Week16/left2.avi"
    vr_path = "Week16/right2.avi"
    
    vl = cv2.VideoCapture(vl_path)
    vr = cv2.VideoCapture(vr_path)

    while True:
        if not paused:
            retl, framel = vl.read()
            retr, framer = vr.read()
            if not retl or not retr:
                print("End of video")
                break

        framer,framel = undistortRectify(framer,framel)
        hsvl = cv2.cvtColor(framel, cv2.COLOR_BGR2HSV)
        hsvr = cv2.cvtColor(framer,cv2.COLOR_BGR2HSV)

        mask_greenL = masker(hsvl, green_lower, green_upper)
        mask_redL = masker(hsvl, red_lower, red_upper)
        mask_greenR = masker(hsvr, green_lower, green_upper)
        mask_redR = masker(hsvr, red_lower, red_upper)

        contours_greenL, _ = cv2.findContours(mask_greenL.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_redL, _ = cv2.findContours(mask_redL.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        green_dot_centersL = calculate_centers(contours_greenL)
        red_dot_centersL = calculate_centers(contours_redL)

        contours_greenR, _ = cv2.findContours(mask_greenR.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_redR, _ = cv2.findContours(mask_redR.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        green_dot_centersR = calculate_centers(contours_greenR)
        red_dot_centersR = calculate_centers(contours_redR)

        #filter_centers(red_dot_centersL, "Red","l", 20, index)
        #filter_centers(green_dot_centersL, "Green","l", 20, index)
        #filter_centers(red_dot_centersR,"Red","r",20,index)
        #filter_centers(green_dot_centersR,"Green","r",20,index)

        for center in green_dot_centersL:
            cv2.circle(framel, center, 3, (0, 255, 0), -1)
        for center in red_dot_centersL:
            cv2.circle(framel, center, 3, (255, 255, 255), -1)

        for center in green_dot_centersR:
            cv2.circle(framer, center, 3, (0, 255, 0), -1)
        for center in red_dot_centersR:
            cv2.circle(framer, center, 3, (255, 255, 255), -1)

        #name = f"./Week15/Frameres/framer{index}.png"
        #cv2.imwrite(name, framer)
    
                    
        #depthsRed = calculate_depths(framer,framel,B,f_pixel,alpha)
        #depthsGreen = calculate_depths(framer,framel,B,f_pixel,alpha,"Week16/Green/left","Week16/Green/right")
        #print(depthsGreen)
        
        framel,framer, depthRed = displaydepth(framel,framer,red_dot_centersL,red_dot_centersR,B,f_pixel,alpha)
        framel,framer, depthGreen = displaydepth(framel,framer,green_dot_centersL,green_dot_centersR,B,f_pixel,alpha)
        print(depthGreen,depthRed)
    
        cv2.imshow("Left",framel)
        cv2.imshow("Right",framer)

        index += 1

        key = cv2.waitKey(1) & 0xFF
        if key == ord("p"):
            paused = not paused
            print("Paused" if paused else "Resumed")
        elif key == ord("d"):
            break

    vl.release()
    vr.release()

    cv2.destroyAllWindows()


# Stereo vision setup parameters
frame_rate = 30     #Camera frame rate
B = 2.7             #Distance between the cameras [cm]
f = 3.67            #Camera lense's focal length [mm]
alpha = 78          #Camera field of view in the horisontal plane [degrees]
f_pixel = f * (1080/((1/2.88)*25.4))
#Camera focal length is 3.67mm for 920 pro and 78 degrees camera field of view
#For c930 it is approximately the same and 90 degrees camera field of view

if __name__ == "__main__":
    main()

In [ ]:

def main():
    red_lower = np.array([0, 100, 122], np.uint8)
    red_upper = np.array([180, 255, 255], np.uint8)
    green_lower = np.array([35, 20, 16], np.uint8)
    green_upper = np.array([110, 196, 197], np.uint8)

    
    paused = False
    index = 0

    vl_path = "Week16/left2.avi"
    vr_path = "Week16/right2.avi"
    
    vl = cv2.VideoCapture(vl_path)
    vr = cv2.VideoCapture(vr_path)

    f = open("Week16/RedAllLeft.txt","w")
    f2 = open("Week16/RedAllRight.txt","w")
    f3 = open("Week16/GreenAllRight.txt","w")
    f4= open("Week16/GreenAllLeft.txt","w")

    while True:
        if not paused:
            retl, framel = vl.read()
            retr, framer = vr.read()
            if not retl or not retr:
                print("End of video")
                break

        framer,framel = undistortRectify(framer,framel)
        hsvl = cv2.cvtColor(framel, cv2.COLOR_BGR2HSV)
        hsvr = cv2.cvtColor(framer,cv2.COLOR_BGR2HSV)

        mask_greenL = masker(hsvl, green_lower, green_upper)
        mask_redL = masker(hsvl, red_lower, red_upper)
        mask_greenR = masker(hsvr, green_lower, green_upper)
        mask_redR = masker(hsvr, red_lower, red_upper)

        contours_greenL, _ = cv2.findContours(mask_greenL.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_redL, _ = cv2.findContours(mask_redL.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        green_dot_centersL = calculate_centers(contours_greenL)
        red_dot_centersL = calculate_centers(contours_redL)

        contours_greenR, _ = cv2.findContours(mask_greenR.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_redR, _ = cv2.findContours(mask_redR.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        green_dot_centersR = calculate_centers(contours_greenR)
        red_dot_centersR = calculate_centers(contours_redR)

        #filter_centers(red_dot_centersL, "Red","l", 20, index)
        #filter_centers(green_dot_centersL, "Green","l", 20, index)
        #filter_centers(red_dot_centersR,"Red","r",20,index)
        #filter_centers(green_dot_centersR,"Green","r",20,index)

        for center in green_dot_centersL:
            cv2.circle(framel, center, 3, (0, 255, 0), -1)
        for center in red_dot_centersL:
            cv2.circle(framel, center, 3, (255, 255, 255), -1)

        for center in green_dot_centersR:
            cv2.circle(framer, center, 3, (0, 255, 0), -1)
        for center in red_dot_centersR:
            cv2.circle(framer, center, 3, (255, 255, 255), -1)

        #name = f"./Week15/Frameres/framer{index}.png"
        #cv2.imwrite(name, framer)
    
                    
        #depthsRed = calculate_depths(framer,framel,B,f_pixel,alpha)
        #depthsGreen = calculate_depths(framer,framel,B,f_pixel,alpha,"Week16/Green/left","Week16/Green/right")
        #print(depthsGreen)
        
        framel,framer, depthRed = displaydepth(framel,framer,red_dot_centersL,red_dot_centersR,B,f_pixel,alpha)
        framel,framer, depthGreen = displaydepth(framel,framer,green_dot_centersL,green_dot_centersR,B,f_pixel,alpha)
        print(depthGreen,depthRed)

        #with open("Week16/Depth.txt","a") as file2:
        #    print(depthGreen,",",depthRed,file=file2)
        left_frame_line_r = " ".join([f"{x},{y}" for x, y in red_dot_centersL])
        right_frame_line_r = ' '.join([f"{x},{y}" for x, y in red_dot_centersR])
        left_frame_line_g = " ".join([f"{x},{y}" for x, y in green_dot_centersL])
        right_frame_line_g = ' '.join([f"{x},{y}" for x, y in green_dot_centersR])
        f.write(f"{left_frame_line_r}\n")
        f2.write(f"{right_frame_line_r}\n")
        f3.write(f"{right_frame_line_g}\n")
        f4.write(f"{left_frame_line_g}\n")

        index += 1

        key = cv2.waitKey(1) & 0xFF
        if key == ord("p"):
            paused = not paused
            print("Paused" if paused else "Resumed")
        elif key == ord("d"):
            break

    f.close()
    f2.close()
    f3.close()
    f4.close()

    vl.release()
    vr.release()

    cv2.destroyAllWindows()


# Stereo vision setup parameters
frame_rate = 30     #Camera frame rate
B = 2.7             #Distance between the cameras [cm]
f = 3.67            #Camera lense's focal length [mm]
alpha = 78          #Camera field of view in the horisontal plane [degrees]
f_pixel = f * (1080/((1/2.88)*25.4))
#Camera focal length is 3.67mm for 920 pro and 78 degrees camera field of view
#For c930 it is approximately the same and 90 degrees camera field of view

if __name__ == "__main__":
    main()